# Week 3 project

In [1]:
!pip install folium

     |████████████████████████████████| 92kB 8.2MB/s eta 0:00:011


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium
import json

## Get data from website 

In [3]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

## Parse response data and prepare dataframe

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all(class_='wikitable')[0]
data = [[td.get_text().strip() for td in tr.find_all("td")] for tr in table.find_all("tr")]

## Assign data to dataframe and do cleaning

In [5]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[df['Borough'] != 'Not assigned']
df = df[df['Borough'].notnull()]

df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


## Join neighbourhood to post code and borough

In [6]:
df_joined = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).to_frame().reset_index()
df_joined.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_joined.shape

(103, 3)

## Load geo data 

In [8]:
!wget -q https://cocl.us/Geospatial_data

In [9]:
df_location = pd.read_csv('Geospatial_data')
df_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Add geo data to orignal data frame by using post code

In [10]:
df_combined = df_joined.join(df_location.set_index('Postal Code'), on='PostalCode')
df_combined.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Fetch venue data from foursquare

### Select Toronto

In [12]:
df_toronto = df_combined[df_combined['Borough'].str.contains('Toronto')]
df_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [13]:
df_toronto.shape

(39, 5)

In [14]:
CLIENT_ID = 'J1JVHYT054FYK125LDRYSJGY21WREMQ1YPMOWHXVOWYQRZZG' 
CLIENT_SECRET = 'VFQVG2OMAHMJWDBWWJFYLXQGJ4AKFVOBHAVUJ0ISCLPYMZ3K'
VERSION = '20190425'
url = 'https://api.foursquare.com/v2/venues/search'

In [15]:
import json
for index, item in df_combined.iterrows():

    params = dict(
      client_id=CLIENT_ID,
      client_secret=CLIENT_SECRET,
      v=VERSION,
      ll=f"{item['Latitude']},{item['Longitude']}",
      radius=500,
      limit=100
    )
    results = requests.get(url, params=params).json()['response']['venues']
    print(json.dumps(results))
    print(item)
    break

[{"id": "4e389459b61c438b547e8ed4", "name": "Rouge Park - Woodland Trail", "location": {"lat": 43.80178186082014, "lng": -79.20042735921538, "labeledLatLngs": [{"label": "display", "lat": 43.80178186082014, "lng": -79.20042735921538}], "distance": 732, "cc": "CA", "country": "Canada", "formattedAddress": ["Canada"]}, "categories": [{"id": "4bf58dd8d48988d159941735", "name": "Trail", "pluralName": "Trails", "shortName": "Trail", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_", "suffix": ".png"}, "primary": true}], "referralId": "v-1583694260", "hasPerk": false}, {"id": "4bb8cc383db7b713099f219a", "name": "Shell", "location": {"address": "8301 Sheppard Ave E", "lat": 43.80322660320925, "lng": -79.19241398073378, "labeledLatLngs": [{"label": "display", "lat": 43.80322660320925, "lng": -79.19241398073378}], "distance": 415, "postalCode": "M1B 2Y7", "cc": "CA", "city": "Toronto", "state": "ON", "country": "Canada", "formattedAddress": ["8301 Sheppard 